# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 21
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr21.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr21.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 62 62

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000155307,ENSG00000180530,ENSG00000154719,ENSG00000154721,ENSG00000154723,ENSG00000142192,ENSG00000154734,ENSG00000156261,ENSG00000171189,ENSG00000156299
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000155307,1.0000000,0.1193705,0.00000000,0.000000000,0.000000000,0.00000000,0.000000000,0.00000000,0.00000000,0.0000000
ENSG00000180530,0.1193705,1.0000000,0.00000000,0.000000000,0.000000000,0.00000000,0.000000000,0.00000000,0.00000000,0.0000000
ENSG00000154719,0.0000000,0.0000000,1.00000000,0.148486926,-0.025066736,-0.10116429,0.037798657,-0.64329083,-0.24955390,0.0000000
ENSG00000154721,0.0000000,0.0000000,0.14848693,1.000000000,-0.004689488,-0.08914295,0.035862540,-0.17118742,0.01094083,0.0000000
ENSG00000154723,0.0000000,0.0000000,-0.02506674,-0.004689488,1.000000000,-0.05914889,0.057730643,0.04656161,0.06493727,0.0000000
ENSG00000142192,0.0000000,0.0000000,-0.10116429,-0.089142948,-0.059148892,1.00000000,-0.003978145,-0.11120360,-0.04971146,-0.1166378


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 1

[1] -0.02632186

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 62 62

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000155307,ENSG00000180530,ENSG00000154719,ENSG00000154721,ENSG00000154723,ENSG00000142192,ENSG00000154734,ENSG00000156261,ENSG00000171189,ENSG00000156299
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000155307,1.0000000,0.1193705,0.00000000,0.000000000,0.000000000,0.00000000,0.000000000,0.00000000,0.00000000,0.0000000
ENSG00000180530,0.1193705,1.0000000,0.00000000,0.000000000,0.000000000,0.00000000,0.000000000,0.00000000,0.00000000,0.0000000
ENSG00000154719,0.0000000,0.0000000,1.00000000,0.148486926,-0.025066736,-0.10116429,0.037798657,-0.64329083,-0.24955390,0.0000000
ENSG00000154721,0.0000000,0.0000000,0.14848693,1.000000000,-0.004689488,-0.08914295,0.035862540,-0.17118742,0.01094083,0.0000000
ENSG00000154723,0.0000000,0.0000000,-0.02506674,-0.004689488,1.000000000,-0.05914889,0.057730643,0.04656161,0.06493727,0.0000000
ENSG00000142192,0.0000000,0.0000000,-0.10116429,-0.089142948,-0.059148892,1.00000000,-0.003978145,-0.11120360,-0.04971146,-0.1166378


,ENSG00000155307,ENSG00000180530,ENSG00000154719,ENSG00000154721,ENSG00000154723,ENSG00000142192,ENSG00000154734,ENSG00000156261,ENSG00000171189,ENSG00000156299
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000155307,1.00000000,0.05632358,0.0000000,0.000000000,0.000000000,0.00000000,0.00000000,0.000000000,0.000000000,0.00000000
ENSG00000180530,0.05632358,1.00000000,0.0000000,0.000000000,0.000000000,0.00000000,0.00000000,0.000000000,0.000000000,0.00000000
ENSG00000154719,0.00000000,0.00000000,1.0000000,0.120312748,-0.023928997,-0.01410330,0.04349733,-0.229381937,-0.043082110,-0.01708879
ENSG00000154721,0.00000000,0.00000000,0.1203127,1.000000000,0.002276596,-0.03598078,0.02857111,-0.110291606,0.006284277,0.01745799
ENSG00000154723,0.00000000,0.00000000,-0.0239290,0.002276596,1.000000000,-0.03237293,0.01612258,0.025944244,0.021264737,0.02438941
ENSG00000142192,0.00000000,0.00000000,-0.0141033,-0.035980776,-0.032372925,1.00000000,-0.02126774,-0.002526252,-0.027752791,-0.06286539


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)